In [1]:
import os
from dotenv import load_dotenv
import sqlite3
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box

In [2]:
# Load environment variables
load_dotenv()

console = Console()

In [3]:
DB_FILE = os.getenv("DB_FILE", "game_database.db")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [4]:
def get_connection():
    try:
        os.makedirs(os.path.dirname(DB_FILE), exist_ok=True)
        conn = sqlite3.connect(DB_FILE, timeout=10)
        cursor = conn.cursor()
        cursor.execute("PRAGMA foreign_keys = ON;")
        cursor.close()
        return conn
    except Exception as e:
        console.print(f"[red]❌ Could not open DB connection: {e}[/red]")
        raise


In [5]:
def execute_query(query: str):
    conn = None
    cursor = None
    try:
        conn = get_connection()
        cursor = conn.cursor()

        console.print("[bold cyan]📡 Executing Query...[/bold cyan]")
        cursor.execute(query)

        if query.strip().lower().startswith("select"):
            rows = cursor.fetchall()
            if rows:
                columns = [description[0] for description in cursor.description]

                table = Table(title="Query Results", box=box.ROUNDED, show_lines=True)
                for col in columns:
                    table.add_column(col, style="bold magenta")

                for row in rows:
                    table.add_row(*[str(cell) for cell in row])

                console.print(table)
                console.print(f"✅ [green]{len(rows)} rows fetched successfully.[/green]")
            else:
                console.print(Panel("[yellow]⚠️ Query returned no rows.[/yellow]", title="No Data", style="yellow"))
        else:
            conn.commit()
            console.print(Panel(f"✅ [green]Query executed successfully.[/green]", title="Success", style="green"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error executing query:[/red] {str(e)}", title="Error", style="red"))

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")


In [44]:
def list_tables():
    """List all tables with row counts."""
    query = """
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name NOT LIKE 'sqlite_%'
    ORDER BY name;
    """
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(query)
        tables = cursor.fetchall()

        if not tables:
            console.print(Panel("[yellow]⚠️ No tables found in the database.[/yellow]", title="No Tables", style="yellow"))
            return

        table = Table(title="Tables in Database", box=box.ROUNDED, show_lines=True)
        table.add_column("Table Name", style="bold green")
        table.add_column("Number of Records", style="bold cyan", justify="right")

        for (table_name,) in tables:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            table.add_row(f"📋 {table_name}", str(count))

        console.print(table)
    except Exception as e:
        console.print(Panel(f"❗ [red]Error listing tables:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [6]:
def table_info(table_name: str):
    """Show info about table: columns, row count, and sample rows (max 10)."""
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Check if table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not cursor.fetchone():
            console.print(Panel(f"⚠️ Table '{table_name}' does not exist.", style="yellow"))
            return

        # Get columns info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = cursor.fetchall()
        columns = [col[1] for col in columns_info]

        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]

        # Print columns and row count
        info_panel = Panel.fit(
            f"📋 [bold green]Table:[/bold green] {table_name}\n"
            f"🔢 [bold cyan]Columns:[/bold cyan] {', '.join(columns)}\n"
            f"📊 [bold magenta]Total Rows:[/bold magenta] {row_count}",
            title="Table Info",
            style="blue",
        )
        console.print(info_panel)

        # Fetch and show up to 10 rows
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
        rows = cursor.fetchall()

        if rows:
            table = Table(title=f"Sample Data from {table_name}", box=box.ROUNDED, show_lines=True)
            for col in columns:
                table.add_column(col, style="bold magenta")
            for row in rows:
                table.add_row(*[str(cell) for cell in row])
            console.print(table)
        else:
            console.print(Panel("[yellow]⚠️ No data found in the table.[/yellow]", title="Empty Table", style="yellow"))

    except Exception as e:
        console.print(Panel(f"❗ [red]Error fetching table info:[/red] {str(e)}", title="Error", style="red"))
    finally:
        cursor.close()
        conn.close()
        console.print("[bold cyan]🔌 Connection closed.[/bold cyan]\n")

In [7]:
list_tables()

NameError: name 'list_tables' is not defined

In [8]:
def drop_tables_interactive():
    try:
        conn = get_connection()
        cursor = conn.cursor()

        # Get list of user tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            print("No user tables found to drop.")
            return

        print("\n📋 Existing tables:")
        for idx, table in enumerate(tables, 1):
            print(f"{idx}. {table}")

        print("\nOptions:")
        print("  🔢 Enter the table number to drop a specific table")
        print("  💣 Enter 'all' to drop ALL tables")
        print("  ❌ Enter 'exit' to cancel and exit")

        while tables:
            choice = input("\nYour choice: ").strip().lower()

            if choice == 'exit':
                print("🚪 Exit without dropping tables.")
                break

            elif choice == 'all':
                for table in tables:
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        print(f"✅ Dropped table '{table}'")
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")
                conn.commit()
                break

            elif choice.isdigit():
                idx = int(choice)
                if 1 <= idx <= len(tables):
                    table = tables[idx - 1]
                    try:
                        cursor.execute(f"DROP TABLE IF EXISTS {table};")
                        conn.commit()
                        print(f"✅ Dropped table '{table}'")
                        tables.pop(idx - 1)
                    except sqlite3.OperationalError as e:
                        print(f"❌ Failed to drop '{table}': {e}")

                    if not tables:
                        print("📭 No more tables left.")
                        break
                    else:
                        print("\n📋 Remaining tables:")
                        for i, t in enumerate(tables, 1):
                            print(f"{i}. {t}")
                else:
                    print("⚠️ Invalid table number, try again.")
            else:
                print("⚠️ Invalid input, try again.")

    except sqlite3.OperationalError as e:
        print(f"🚫 Database operation failed: {e}")
    finally:
        try:
            cursor.close()
        except:
            pass
        try:
            conn.close()
        except:
            pass


In [18]:
drop_tables_interactive()

No user tables found to drop.


## 1.1 🧑‍💼 1. Players Table

In [10]:
query_two_legged = """
CREATE TABLE two_legged_ties (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    round_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    away_team_id INTEGER NOT NULL,
    stage_id INTEGER,
    leg1_match_id INTEGER NOT NULL,  -- FK to matches.id for first leg
    leg2_match_id INTEGER NOT NULL,  -- FK to matches.id for second leg
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (round_id) REFERENCES rounds(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    FOREIGN KEY (home_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (away_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (stage_id) REFERENCES stages(id) ON DELETE CASCADE,
    FOREIGN KEY (leg1_match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (leg2_match_id) REFERENCES matches(id) ON DELETE CASCADE
);
"""
execute_query(query_two_legged)


📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: table two_legged_ties already exists                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [ ]:
query_players = """
CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    avatar_name TEXT DEFAULT 'Assets\\Avatars\\No-One.jpg',  -- escaped correctly
    role TEXT CHECK(role IN ('admin', 'player')) DEFAULT 'player',
    is_confirmed INTEGER DEFAULT 1,
    reset_token TEXT,
    score INTEGER DEFAULT 0,
    bonous INTEGER DEFAULT 0, 
    total_leagues_won INTEGER DEFAULT 0,
    total_cups_won INTEGER DEFAULT 0,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    last_login_at DATETIME DEFAULT NULL
);
"""
execute_query(query_players)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 2.2 🏆 2. Leagues Table

In [73]:
query_leagues = """
-- 🌍 Leagues Table
CREATE TABLE IF NOT EXISTS leagues (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE NOT NULL,
    code TEXT UNIQUE,                         -- e.g., "PL", "CL"
    country TEXT,
    emblem TEXT,                              -- URL or logo filename
    type TEXT CHECK(type IN ('LEAGUE', 'CUP')) DEFAULT 'LEAGUE',
    founded_year INTEGER,
    is_active INTEGER DEFAULT 1,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    logo_path TEXT NOT NULL
);
"""
execute_query(query_leagues)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 3.3 📅 3. Rounds Table

In [74]:
query_rounds = """
CREATE TABLE IF NOT EXISTS rounds (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,                       -- e.g., Round1, Round 2
    round_number INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    prediction_deadline DATETIME NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(query_rounds)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 4.4 🏟️ 4. Teams Table

In [76]:
query_teams = """
CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY,  -- Use API team ID here
    api_id INTEGER,                           -- Optional: ID from external API like Football-Data
    name TEXT NOT NULL UNIQUE,
    short_name TEXT,
    tla TEXT,                                 -- Three-letter acronym like LIV
    official_name TEXT NOT NULL,
    logo_path TEXT,
    nationality TEXT,
    Venue_name TEXT DEFAULT no_data,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
"""

execute_query(query_teams)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## team_league join table

In [26]:
execute_query("""
CREATE TABLE IF NOT EXISTS team_league (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    team_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    season TEXT,  -- Optional (e.g., "2024/2025")
    joined_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    UNIQUE(team_id, league_id, season)  -- Prevent duplicates
);
""")



📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 5.5 ⚽ 5. Matches Table

In [81]:
query_matches = """
CREATE TABLE IF NOT EXISTS matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    round_id INTEGER NOT NULL,
    league_id INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    away_team_id INTEGER NOT NULL,
    match_datetime DATETIME NOT NULL,  -- Stored in UTC
    status TEXT CHECK (status IN ('upcoming', 'live', 'finished', 'cancelled')) DEFAULT 'upcoming',
    home_score INTEGER DEFAULT NULL,
    away_score INTEGER DEFAULT NULL,
    stage_id INTEGER,  -- Optional, nullable
    penalty_winner INTEGER DEFAULT NULL,
    is_predictable INTEGER DEFAULT 0,  -- 0: Not predictable, 1: Predictable
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    matchday INT NOT NULL,
    Venue_Name TEXT DEFAULT no_data,
    api_match_id INTEGER UNIQUE, 
    FOREIGN KEY (round_id) REFERENCES rounds(id) ON DELETE CASCADE,
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    FOREIGN KEY (home_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (away_team_id) REFERENCES teams(id) ON DELETE CASCADE,
    FOREIGN KEY (stage_id) REFERENCES stages(id) ON DELETE CASCADE,
    FOREIGN KEY (penalty_winner) REFERENCES teams(id) ON DELETE SET NULL
);

"""
execute_query(query_matches)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 6.6 🔁 6. Stages Table

In [79]:
query_stages = """
CREATE TABLE IF NOT EXISTS stages (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    league_id INTEGER NOT NULL,
    is_two_legged INTEGER DEFAULT 0 CHECK (is_two_legged IN (0, 1)),
    allows_draw INTEGER DEFAULT 1 CHECK (allows_draw IN (0, 1)),
    has_penalties INTEGER DEFAULT 0 CHECK (has_penalties IN (0, 1)),
    FOREIGN KEY (league_id) REFERENCES leagues(id) ON DELETE CASCADE,
    UNIQUE(name, league_id)
);
"""
execute_query(query_stages)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [31]:
query_legs = """
CREATE TABLE IF NOT EXISTS two_legged_ties (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_leg_match_id INTEGER NOT NULL,
    second_leg_match_id INTEGER NOT NULL,
    winner_team_id INTEGER,  -- Nullable until calculated
    FOREIGN KEY (first_leg_match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (second_leg_match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (winner_team_id) REFERENCES teams(id) ON DELETE SET NULL,
    UNIQUE (first_leg_match_id, second_leg_match_id)
);
"""
execute_query(query_legs)


📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

## 8.8 predictions Table

In [80]:
query_predictions = """
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER NOT NULL,
    match_id INTEGER NOT NULL,

    predicted_home_score INTEGER NOT NULL,
    predicted_away_score INTEGER NOT NULL,
    predicted_penalty_winner_id INTEGER,  -- FK to teams.id (nullable if no penalties)

    score INTEGER DEFAULT 0,  -- Computed after match ends

    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,

    FOREIGN KEY (player_id) REFERENCES players(id) ON DELETE CASCADE,
    FOREIGN KEY (match_id) REFERENCES matches(id) ON DELETE CASCADE,
    FOREIGN KEY (predicted_penalty_winner_id) REFERENCES teams(id) ON DELETE SET NULL,

    UNIQUE (player_id, match_id)  -- Player can only predict once per match
);
"""
execute_query(query_predictions)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [73]:
list_tables()

            Tables in Database            
╭────────────────────┬───────────────────╮
│ Table Name         │ Number of Records │
├────────────────────┼───────────────────┤
│ 📋 achievements    │                 0 │
├────────────────────┼───────────────────┤
│ 📋 avatars         │                 0 │
├────────────────────┼───────────────────┤
│ 📋 leagues         │                13 │
├────────────────────┼───────────────────┤
│ 📋 legs            │                 4 │
├────────────────────┼───────────────────┤
│ 📋 matches         │                10 │
├────────────────────┼───────────────────┤
│ 📋 players         │                 2 │
├────────────────────┼───────────────────┤
│ 📋 predictions     │                 7 │
├────────────────────┼───────────────────┤
│ 📋 rounds          │                 0 │
├────────────────────┼───────────────────┤
│ 📋 stages          │               100 │
├────────────────────┼───────────────────┤
│ 📋 team_league     │               113 │
├────────────────────┼───────────────────┤
│ 📋 teams           │                57 │
├────────────────────┼───────────────────┤
│ 📋 two_legged_ties │                 0 │
╰────────────────────┴───────────────────╯

🔌 Connection closed.

In [37]:
table_info("two_legged_ties")

╭────────────────────────────── Table Info ───────────────────────────────╮
│ 📋 Table: two_legged_ties                                               │
│ 🔢 Columns: id, first_leg_match_id, second_leg_match_id, winner_team_id │
│ 📊 Total Rows: 0                                                        │
╰─────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [38]:
table_info("matches")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id, penalty_winner, created_at, updated_at                                                    │
│ 📊 Total Rows: 0                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Empty Table ──────────────────────────────────────────────────╮
│ ⚠️ No data found in the table.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [90]:
import sqlite3

conn = sqlite3.connect("game_database.db")
cur = conn.cursor()

try:
    cur.execute("ALTER TABLE players ADD COLUMN avatar_url TEXT")
except sqlite3.OperationalError as e:
    print("Column probably already exists:", e)

conn.commit()
conn.close()


In [76]:
def show_and_clear_teams():
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch all records from teams
    cursor.execute("SELECT * FROM matches;")
    records = cursor.fetchall()
    print("Records before deletion:")
    if records:
        for row in records:
            print(row)
    else:
        print("No records found.")

    # Delete all records from teams
    cursor.execute("DELETE FROM matches;")
    conn.commit()

    # Fetch again to confirm deletion
    cursor.execute("SELECT * FROM matches;")
    records_after = cursor.fetchall()
    print("\nRecords after deletion:")
    if records_after:
        for row in records_after:
            print(row)
    else:
        print("No records found.")

    cursor.close()
    conn.close()


# Real Data

In [57]:
import os
import requests

# Constants
API_TOKEN = '4c936cda044a4c2eaa3ed158b90e66d3'  # Replace with your actual API key
BASE_URL = 'https://api.football-data.org/v4'
HEADERS = {'X-Auth-Token': API_TOKEN}
LOGO_DOWNLOAD_PATH = r'D:\Football_Prediction_Game\assets\clubs\Team'

# Ensure the logo directory exists
os.makedirs(LOGO_DOWNLOAD_PATH, exist_ok=True)

def get_leagues():
    """Fetches a list of top-tier football leagues."""
    url = f'{BASE_URL}/competitions'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        # Only return popular leagues
        popular_league_codes = ['PL', 'PD', 'BL1', 'SA', 'FL1', 'DED', 'PPL']
        leagues = [comp for comp in data['competitions'] if comp['code'] in popular_league_codes]
        return leagues
    else:
        print(f"Error fetching leagues: {response.status_code} - {response.text}")
        return []


def get_teams(league_id):
    """Fetches teams for a given league ID."""
    url = f'{BASE_URL}/competitions/{league_id}/teams?season=2024'
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        return data['teams']
    else:
        print(f"Error fetching teams: {response.status_code}")
        return []

def download_logo(team_name, logo_url):
    """Downloads the team's logo to the specified directory."""
    try:
        response = requests.get(logo_url, stream=True)
        if response.status_code == 200:
            # Sanitize team name for filename
            filename = f"{team_name.replace(' ', '_').replace('/', '_')}.png"
            filepath = os.path.join(LOGO_DOWNLOAD_PATH, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Downloaded logo for {team_name}")
        else:
            print(f"Failed to download logo for {team_name}: {response.status_code}")
    except Exception as e:
        print(f"Error downloading logo for {team_name}: {e}")

def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            print(f"Fetching teams for {selected_league['name']}...")
            teams = get_teams(selected_league['id'])
            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # use 'crest' instead of 'crestUrl' in v4
                    teams_dict[team_name] = logo_url
                print("\nTeam Logos Dictionary:")
                for name, url in teams_dict.items():
                    print(f"{name}: {url}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)
Fetching teams for Premier League...

Team Logos Dictionary:
Arsenal FC: https://crests.football-data.org/57.png
Aston Villa FC: https://crests.football-data.org/58.png
Chelsea FC: https://crests.football-data.org/61.png
Everton FC: https://crests.football-data.org/62.png
Fulham FC: https://crests.football-data.org/63.png
Liverpool FC: https://crests.football-data.org/64.png
Manchester City FC: https://crests.football-data.org/65.png
Manchester United FC: https://crests.football-data.org/66.png
Newcastle United FC: https://crests.football-data.org/67.png
Tottenham Hotspur FC: https://crests.football-data.org/73.png
Wolverhampton Wanderers FC: https://crests.football-data.org/76.png
Leicester City FC: https://crests.football-data.org/338.png
Southampton FC: https://crests.football-data.org/340.pn

In [64]:
def main():
    leagues = get_leagues()
    if not leagues:
        print("No leagues available.")
        return

    # Display leagues
    print("Available Leagues:")
    for idx, league in enumerate(leagues, start=1):
        print(f"{idx}. {league['name']} ({league['area']['name']})")

    # User selects a league
    try:
        choice = int(input("Select a league by number: "))
        if 1 <= choice <= len(leagues):
            selected_league = leagues[choice - 1]
            league_name_clean = selected_league['name'].replace(' ', '_').replace('/', '_')
            print(f"\nFetching teams for {selected_league['name']}...\n")
            teams = get_teams(selected_league['id'])

            if teams:
                teams_dict = {}
                for team in teams:
                    team_name = team['name']
                    logo_url = team.get('crest')  # API v4 uses 'crest'
                    teams_dict[team_name] = logo_url
                    print(f"{team_name}: {logo_url}")

                # Save to text file
                file_path = os.path.join(LOGO_DOWNLOAD_PATH, f"{league_name_clean}.txt")
                with open(file_path, 'w', encoding='utf-8') as f:
                    for name, url in teams_dict.items():
                        f.write(f"{name}: {url}\n")

                print(f"\nSaved teams to {file_path}")
                return teams_dict
            else:
                print("No teams found for this league.")
        else:
            print("Invalid selection.")
    except ValueError:
        print("Please enter a valid number.")


if __name__ == "__main__":
    main()


Available Leagues:
1. Premier League (England)
2. Ligue 1 (France)
3. Bundesliga (Germany)
4. Serie A (Italy)
5. Eredivisie (Netherlands)
6. Primeira Liga (Portugal)
7. Primera Division (Spain)

Fetching teams for Primera Division...

Athletic Club: https://crests.football-data.org/77.png
Club Atlético de Madrid: https://crests.football-data.org/78.png
CA Osasuna: https://crests.football-data.org/79.png
RCD Espanyol de Barcelona: https://crests.football-data.org/80.png
FC Barcelona: https://crests.football-data.org/81.png
Getafe CF: https://crests.football-data.org/82.png
Real Madrid CF: https://crests.football-data.org/86.png
Rayo Vallecano de Madrid: https://crests.football-data.org/87.png
RCD Mallorca: https://crests.football-data.org/89.png
Real Betis Balompié: https://crests.football-data.org/90.png
Real Sociedad de Fútbol: https://crests.football-data.org/92.png
Villarreal CF: https://crests.football-data.org/94.png
Valencia CF: https://crests.football-data.org/95.png
Real Vallad

In [8]:
import sqlite3

def fetch_all(query, params=()):
    conn = get_connection()
    conn.row_factory = sqlite3.Row  # Enables dict-style row access
    cur = conn.cursor()
    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()
    return [dict(row) for row in rows]

def execute_query(query, params=()):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute(query, params)
    conn.commit()
    conn.close()


In [9]:
def assign_team_to_leagues_console():
    # Fetch all teams
    teams = fetch_all("SELECT id, name FROM teams")
    if not teams:
        print("❌ No teams found.")
        return

    print("\n📋 List of Teams:")
    for i, team in enumerate(teams, start=1):
        print(f"{i}. {team['name']}")

    choice = int(input("\nEnter the number of the team to assign leagues: "))
    selected_team = teams[choice - 1]
    selected_team_id = selected_team['id']

    # Fetch all leagues
    leagues = fetch_all("SELECT id, name FROM leagues")
    if not leagues:
        print("❌ No leagues found.")
        return

    print("\n🏆 Available Leagues:")
    for i, league in enumerate(leagues, start=1):
        print(f"{i}. {league['name']}")

    selected_numbers = input("\nEnter the numbers of the leagues to assign (comma-separated): ")
    selected_indices = [int(x.strip()) for x in selected_numbers.split(",")]
    selected_league_ids = [leagues[i - 1]['id'] for i in selected_indices]

    # Delete old assignments
    execute_query("DELETE FROM team_league WHERE team_id = ?", (selected_team_id,))

    # Insert new assignments
    for league_id in selected_league_ids:
        execute_query(
            "INSERT INTO team_league (team_id, league_id) VALUES (?, ?)",
            (selected_team_id, league_id)
        )

    print(f"\n✅ Successfully updated leagues for team '{selected_team['name']}'.")

assign_team_to_leagues_console()


📋 List of Teams:
1. AFC Bournemouth
2. Al Ahly
3. Al-Ain
4. Al-Hilal
5. Arsenal FC
6. Aston Villa FC
7. Athletic Club
8. Atletico Madrid
9. Barcelona
10. Borussia Dortmund
11. Brentford FC
12. Brighton & Hove Albion FC
13. CA Osasuna
14. Chelsea FC
15. Crystal Palace FC
16. Deportivo Alavés
17. Elche
18. Everton FC
19. FC Bayern München
20. France
21. Fulham FC
22. Germany
23. Getafe CF
24. Girona FC
25. Inter Miami
26. Juventus FC
27. Levante UD
28. Liverpool FC
29. Manchester City FC
30. Manchester United FC
31. Newcastle United FC
32. Nottingham Forest FC
33. Paris Saint-Germain FC
34. Portugal
35. Pyramids
36. RC Celta de Vigo
37. RCD Espanyol
38. RCD Mallorca
39. Rayo Vallecano
40. Real Betis
41. Real Madrid
42. Real Sociedad
43. Sevilla FC
44. Spain
45. Test
46. Tottenham Hotspur FC
47. Valencia CF
48. Villarreal CF
49. West Ham United FC
50. Wolverhampton Wanderers FC
51. Zamalek

🏆 Available Leagues:
1. Bundesliga
2. CAF Champions League
3. Club World Championship
4. Copa del 

In [133]:
table_info("matches")
table_info("rounds")
table_info("two_legged_ties")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: matches                                                                                               │
│ 🔢 Columns: id, round_id, league_id, home_team_id, away_team_id, match_datetime, status, home_score,            │
│ away_score, stage_id, penalty_winner                                                                            │
│ 📊 Total Rows: 6                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from matches                                              
╭────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬─────────╮
│ id │ round_id │ league_… │ home_te… │ away_te… │ match_d… │ status   │ home_sc… │ away_sc… │ stage_id │ penalt… │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 57 │ 25       │ 13       │ 118      │ 121      │ 2025-06… │ finished │ 1        │ 2        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 58 │ 25       │ 13       │ 119      │ 120      │ 2025-06… │ finished │ 4        │ 5        │ 222      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 59 │ 26       │ 13       │ 118      │ 120      │ 2025-06… │ finished │ 0        │ 2        │ 223      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 60 │ 26       │ 13       │ 121      │ 119      │ 2025-06… │ finished │ 2        │ 2        │ 224      │ 121     │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 71 │ 26       │ 6        │ 65       │ 83       │ 2025-06… │ finished │ 2        │ 1        │ 220      │ None    │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 72 │ 26       │ 6        │ 83       │ 65       │ 2025-06… │ finished │ 2        │ 1        │ 220      │ 83      │
╰────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴─────────╯

🔌 Connection closed.

╭────────────────────── Table Info ──────────────────────╮
│ 📋 Table: rounds                                       │
│ 🔢 Columns: id, name, start_date, end_date, created_at │
│ 📊 Total Rows: 2                                       │
╰────────────────────────────────────────────────────────╯

                    Sample Data from rounds                     
╭────┬─────────┬────────────┬────────────┬─────────────────────╮
│ id │ name    │ start_date │ end_date   │ created_at          │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 25 │ Round 1 │ 2025-05-31 │ 2025-06-06 │ 2025-06-10 00:01:30 │
├────┼─────────┼────────────┼────────────┼─────────────────────┤
│ 26 │ Round 2 │ 2025-06-07 │ 2025-06-13 │ 2025-06-10 00:02:24 │
╰────┴─────────┴────────────┴────────────┴─────────────────────╯

🔌 Connection closed.

╭────────────────────────────── Table Info ───────────────────────────────╮
│ 📋 Table: two_legged_ties                                               │
│ 🔢 Columns: id, first_leg_match_id, second_leg_match_id, winner_team_id │
│ 📊 Total Rows: 2                                                        │
╰─────────────────────────────────────────────────────────────────────────╯

                 Sample Data from two_legged_ties                 
╭────┬────────────────────┬─────────────────────┬────────────────╮
│ id │ first_leg_match_id │ second_leg_match_id │ winner_team_id │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 4  │ 69                 │ 70                  │ None           │
├────┼────────────────────┼─────────────────────┼────────────────┤
│ 5  │ 71                 │ 72                  │ 83             │
╰────┴────────────────────┴─────────────────────┴────────────────╯

🔌 Connection closed.

In [100]:
def assign_all_matches_to_round(round_id=47):
    query = f"""
    UPDATE matches
    SET round_id = {round_id};
    """
    try:
        execute_query(query)
        print(f"✅ All matches have been assigned to round_id = {round_id}")
    except Exception as e:
        print(f"❌ Error updating matches: {e}")

assign_all_matches_to_round()

📡 Executing Query...

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ ❗ Error executing query: FOREIGN KEY constraint failed                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

✅ All matches have been assigned to round_id = 47


In [110]:
def delete_all_records(connection):
    """
    Delete all records from the 'two_legged_ties', 'matches', 'stages', and 'rounds' tables.
    
    Args:
        connection: SQLite3 connection object.
    """
    try:
        cursor = connection.cursor()
        # Disable foreign key constraint temporarily to avoid issues during deletion
        cursor.execute("PRAGMA foreign_keys = OFF;")
        
        # Delete in order from dependent to parent tables
        cursor.execute("DELETE FROM two_legged_ties;")
        cursor.execute("DELETE FROM matches;")
        cursor.execute("DELETE FROM stages;")
        cursor.execute("DELETE FROM rounds;")
        
        connection.commit()
        print("All records deleted successfully.")
    except Exception as e:
        print(f"Error deleting records: {e}")
        connection.rollback()
    finally:
        # Re-enable foreign keys
        cursor.execute("PRAGMA foreign_keys = ON;")
conn = get_connection()
delete_all_records(conn)


All records deleted successfully.


In [35]:
query_delete_predictions = "DELETE FROM predictions;"
execute_query(query_delete_predictions)

📡 Executing Query...

╭──────────────────────────────────────────────────── Success ────────────────────────────────────────────────────╮
│ ✅ Query executed successfully.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔌 Connection closed.

In [37]:
table_info("players")

╭────────────────────────────────────────────────── Table Info ───────────────────────────────────────────────────╮
│ 📋 Table: players                                                                                               │
│ 🔢 Columns: id, username, email, password_hash, avatar_path, role, is_confirmed, reset_token, created_at,       │
│ updated_at, last_login_at, avatar_url                                                                           │
│ 📊 Total Rows: 2                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                             Sample Data from players                                              
╭────┬─────────┬─────────┬─────────┬─────────┬────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────╮
│ id │ userna… │ email   │ passwo… │ avatar… │ role   │ is_con… │ reset_… │ create… │ update… │ last_l… │ avatar… │
├────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 8  │ BaselA… │ BaselA… │ b'$2b$… │ assets… │ admin  │ 1       │ None    │ 2025-0… │ 2025-0… │ 2025-0… │ assets… │
│    │         │         │         │         │        │         │         │ 14:00:… │ 15:24:… │         │         │
├────┼─────────┼─────────┼─────────┼─────────┼────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 14 │ Mohamed │ basela… │ b'$2b$… │ None    │ player │ 1       │ None    │ 2025-0… │ 2025-0… │ 2025-0… │ None    │
│    │ Khaled  │         │         │         │        │         │         │ 18:00:… │ 23:13:… │         │         │
╰────┴─────────┴─────────┴─────────┴─────────┴────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────╯

🔌 Connection closed.

In [28]:
con = get_connection()
con.execute("PRAGMA foreign_keys = ON;")